standard importing the menu and nutritional info for the dish in the menu

In [9]:
import pandas as pd
import itertools
import time
import numpy as np
from multiprocess import Pool
MENU_LATEST= r'C:/Users/Penelope/Documents/GitHub/Weplate/backend/backend_data_parsing/babson/master_menu/menu.csv'
NUTRITION_TABLE_LATEST = r'C:/Users/Penelope/Documents/GitHub/Weplate/backend/backend_data_parsing/babson/master_nutrition/nutrition_table.csv'
menu = pd.read_csv(MENU_LATEST)
dish = pd.read_csv(NUTRITION_TABLE_LATEST)
items = eval(menu.loc[2, 'items'])
print(items)

[1208, 1236, 1238, 1223, 1240, 1266, 1293, 1276, 1156, 1182, 1146, 1149, 1172, 1110, 1123, 1102, 1119, 1085, 1077, 1068]


In [69]:
sdf = pd.read_csv(r'C:/Users/Penelope/Documents/GitHub/Weplate/backend/backend/algorithm/result/student_profile/nutrient_limit_male.csv')
DATA = np.zeros(16).transpose()
for student_ind in range(len(sdf)):
    b_protein, b_carbohydrate, b_calories, b_total_fat, b_saturated_fat  = sdf.loc[student_ind, ['protein_lim', 'carbohydrate_lim', 'calories_lim', 'total_fat_lim', 'saturated_fat_lim']].values
    bound_protein = [eval(b_protein)]
    bound_carbohydrate = [eval(b_carbohydrate)]
    bound_calories = [eval(b_calories)]
    bound_total_fat = [eval(b_total_fat)]
    bound_saturated_fat = [eval(b_saturated_fat)]
    
    for menu_ind in range(len((menu))):
        items = eval(menu.loc[menu_ind, 'items'])
        if len(items) > 0:

            dish_nutrients = []
            for i in items:  
                protein, carbohydrate, calories, total_fat, saturated_fat, portion_volume, max_pieces = dish[dish.pk == i][['protein', 'carbohydrate', 'calories', 'total_fat', 'saturated_fat', 'portion_volume', 'max_pieces']].values.squeeze(0)
                dish_nutrients.append(dish_nutrient(protein, carbohydrate, calories, total_fat, saturated_fat, portion_volume, i, max_pieces))
            
            # This is just to create a list of possible combinations of dish_nutrient objects for that particular meal
            combinations = []
            for ind_item, i in enumerate(dish_nutrients):
                item_copy = dish_nutrients.copy()
                item_copy.pop(ind_item)
                a = list(itertools.combinations(item_copy, 2))
                a = [(dish_nutrients[ind_item],) + aa for aa in a]
                combinations.extend(a)

            start_time = time.perf_counter()
            with Pool(8) as p:
                inputs = list(itertools.product(bound_protein, bound_carbohydrate, bound_calories, bound_total_fat, bound_saturated_fat, combinations))
                result = p.map(pick_meal, inputs)
            start_time = time.perf_counter() - start_time

            result = sorted(result, key=lambda x : x[-1])
            name = [[student_ind, menu.loc[menu_ind, 'name'], round(start_time,2), mm] for mm in range(min(27,len(result)))]

            DATA = np.vstack((DATA, np.hstack((name,result[:min(27, len(result))]))))

In [68]:
dfr = pd.DataFrame(DATA, columns=['student_id','name', 'comp_time', 'combination#',
'dish_L', 'dish_s1', 'dish_s2', 'vol_L', 'vol_s1', 'vol_s2', 'protein', 'carbohydrate', 'calories', 'total_fat', 'saturated_fat', 'cost'])
dfr.head(3)
dfr.to_csv('male_may_lsqp.csv')


In [5]:
#create a fake class for rapid testing, rather than trying to integrate with the existing code

class dish_nutrient:
    """
    @params: protein - scalar, value from nutritional table
    @params: carbohydrate - scalar, value from nutritional table
    @params: calories - scalar, value from nutritional table
    @params: total_fat - scalar, value from nutritional table
    @params: saturated_fat - scalar, value from nutritional table
    @params: volume - scalar, portion_volume value from nutritional table 
    @params: pk - scalar, pk of the dish 
    @params: max_pieces - scalar, value from nutritional table
    """
    def __init__(self, protein, carbohydrate, calories, total_fat, saturated_fat, volume, pk, max_pieces):
        self.protein = protein
        self.carbohydrate = carbohydrate
        self.calories = calories
        self.total_fat = total_fat
        self.saturated_fat = saturated_fat
        self.volume = volume
        self.pk = pk
        self.max_pieces = max_pieces
    def normalized_nutrients(self):
        """
        Normalize the nutrients value from the nutrition table
        @return: normalized protein, carbohydrate, calories, total_fat, saturated_fat, pk, max_pieces 
        Note that if it's not a discrete item, the max_pieces is set to int(0)
        """ 
        import numpy as np
        if self.volume > 0:
            return np.array([self.protein/self.volume, 
                             self.carbohydrate/self.volume,
                             self.calories/self.volume,
                             self.total_fat/self.volume,
                             self.saturated_fat/self.volume, int(self.pk),int(0)])
        else:
            return np.array([self.protein/self.volume*-1, 
                             self.carbohydrate/self.volume*-1,
                             self.calories/self.volume*-1,
                             self.total_fat/self.volume*-1,
                             self.saturated_fat/self.volume*-1,  int(self.pk), int(self.max_pieces)])

In [12]:
# This is just to create a list of dish_nutrient class objects for all the dishes in that meal
dish_nutrients = []
for i in items:  
    protein, carbohydrate, calories, total_fat, saturated_fat, portion_volume, max_pieces = dish[dish.pk == i][['protein', 'carbohydrate', 'calories', 'total_fat', 'saturated_fat', 'portion_volume', 'max_pieces']].values.squeeze(0)
    dish_nutrients.append(dish_nutrient(protein, carbohydrate, calories, total_fat, saturated_fat, portion_volume, i, max_pieces))
print(dish_nutrients)


[<__main__.dish_nutrient object at 0x000001B12FE0DF40>, <__main__.dish_nutrient object at 0x000001B12FE0D2E0>, <__main__.dish_nutrient object at 0x000001B12FE0D2B0>, <__main__.dish_nutrient object at 0x000001B12FE0D310>, <__main__.dish_nutrient object at 0x000001B12FE0D340>, <__main__.dish_nutrient object at 0x000001B12FE0D220>, <__main__.dish_nutrient object at 0x000001B12FBA5D30>]


In [13]:
# This is just to create a list of possible combinations of dish_nutrient objects for that particular meal
import itertools
combinations = []
for ind_item, i in enumerate(dish_nutrients):
    item_copy = dish_nutrients.copy()
    item_copy.pop(ind_item)
    a = list(itertools.combinations(item_copy, 2))
    a = [(dish_nutrients[ind_item],) + aa for aa in a]
    combinations.extend(a)
print(len(combinations))

105


In [61]:
import numpy as np
import time 
from scipy.optimize import minimize
from scipy.spatial import distance

def unpack(pick_meal):
    from functools import wraps
    @wraps(pick_meal)
    def wrapper(args):
        if isinstance(args, dict):
            return pick_meal(**args)
        else:
            return pick_meal(*args)
    return wrapper

@unpack
def pick_meal(bound_protein, bound_carbohydrate, bound_calories, bound_total_fat, bound_saturated_fat, combination):
    """
    Calculate the volume of the three different portions 
    @params: combination of the dish_nutrient objects 
    @returns: an array 
    [pk of dish in large, pk of dish in small 1, pk of dish in small 2, 
    volume of dish in large, volume of dish in small 1, volume of dish in small 2, 
    nutrients of all these dishes, performance of the objective functions]
    Note: the order of nutrients here: [protein, carbohydrate, calories, total_fat, saturated_fat]
    Since it's multiprocess, so I have to import packages and dependent functions inside pick_meal
    """ 
    import numpy as np
    from scipy.optimize import minimize
    from scipy.spatial import distance
    
    def my_distance(current_x, bounds, weights_below, weights_above):
        """
        Cost function - how far the particular nutrient is off from the lower-bound or upper-bound
        @params: current_x - array, amount of nutrients that current volume and combination offer
        @params: bounds - array, the lower and upper bounds of particular nutrient
        @params: weights_below - array, the penalty weight for particular nutrient falling below lower-bound
        @params: weights_above - array, the penalty weight for particular nutrient going above upper-bound
        @returns: dist - scalar, which is the summation of how far all nutrients are off from target bound
        """ 
        dist = 0
        for indx,  cx in enumerate(current_x):
            if cx > bounds[indx][0] and cx < bounds[indx][1]:
                dist += 0
            elif cx > bounds[indx][1]:
                dist += weights_above[indx]*(cx - bounds[indx][1])**2
            else:
                dist += weights_below[indx]*(bounds[indx][0] - cx)**2
        return dist

    #for i in combinations: 
    dish_1 = combination[0].normalized_nutrients()[:-2]
    dish_2 = combination[1].normalized_nutrients()[:-2]
    dish_3 = combination[2].normalized_nutrients()[:-2]
    dishes = np.array([dish_1, dish_2, dish_3]).transpose()

    wb = [20,8, 1, 50, 1.5*50]
    wa = [20, 8, 1, 50, 1.5*50]
    x0 = np.zeros(3)
    
    if combination[0].max_pieces == 0: 
        bnd1 = (300, 610)
        x0[0] = 300
    else:
        bnd1 = (0, int(combination[0].max_pieces))
        x0[0] = 1
        
    if combination[1].max_pieces == 0: 
        bnd2 = (50, 270)
        x0[1] = 150
    else:
        bnd2 = (0, int(combination[1].max_pieces))
        x0[1] = 1
        
    if combination[2].max_pieces == 0: 
        bnd3 = (50, 270)
        x0[2] = 100
    else:
        bnd3 = (0, int(combination[2].max_pieces))
        x0[2] = 1
        
    bounds = [bound_protein, bound_carbohydrate, bound_calories, bound_total_fat, bound_saturated_fat]
    bounds_lower = [bound_protein[0], bound_carbohydrate[0], bound_calories[0], bound_total_fat[0], bound_saturated_fat[0]]
    bounds_upper = [bound_protein[1], bound_carbohydrate[1], bound_calories[1], bound_total_fat[1], bound_saturated_fat[1]]

    my_objective = lambda x: my_distance(np.dot(dishes,x) , bounds, wb, wa)
    constraint_lower = lambda x: np.dot(dishes,x) - bounds_lower
    constraint_upper = lambda x: bounds_upper - np.dot(dishes,x)
    bnds = (bnd1,bnd2,bnd3) 
       
    con11 = {'type': 'ineq', 'fun': constraint_lower}
    con12 = {'type': 'ineq', 'fun': constraint_upper}

    cons = ([con11, con12])
    solution = minimize(my_objective,x0, method='SLSQP', bounds=bnds, constraints=cons,options={'maxiter': 5000, 'ftol': 1e-08})
    x = solution.x
    x = [round(xx,0) for xx in x]
    solution = [int(combination[0].pk), int(combination[1].pk), int(combination[2].pk), x[0], x[1], x[2]] +  list(np.round(np.dot(dishes,x),2)) + [round(my_objective(x),2)]
    #solution = [combination[0].pk, combination[1].pk, combination[2].pk, x[0], x[1], x[2], np.dot(dishes,x), my_objective(x)]

    return solution



In [18]:
from multiprocess import Pool
from functools import partial
import time

start_time = time.perf_counter()
# should input the boundary somehow later, but just trying to reduce the complexity in testing
bound_protein = [(26.1, 32.6)]
bound_carbohydrate = [(195.7, 205.5)]
bound_calories = [(817.9, 1106.6)]
bound_total_fat = [(26.7, 32.1)]
bound_saturated_fat = [(0, 10.7)]


with Pool(8) as p:
    inputs = list(itertools.product(bound_protein, bound_carbohydrate, bound_calories, bound_total_fat, bound_saturated_fat, combinations))
    result = p.map(pick_meal, inputs)

result = sorted(result, key=lambda x : x[-1])
print(time.perf_counter() - start_time)
print(result[:27])
#bound_protein, bound_carbohydrate, bound_calories, bound_total_fat, bound_saturated_fat

0.8430725000000621
[[1037, 1014, 1040, 610.0, 3.0, 270.0, 23.878184861446904, 189.67092160210998, 1034.1378260943075, 27.89160904187871, 7.5, 389.52754082130554], [1037, 1021, 1014, 610.0, 225.0, 3.0, 32.62757198167278, 185.48643210982806, 1079.786802373747, 30.74467005934367, 7.5, 834.550956660676], [1037, 1014, 1056, 610.0, 3.0, 111.0, 25.881667709266743, 178.3220788881938, 1031.3481664327862, 32.11413934772685, 7.969170033983127, 2416.9005131865174], [1037, 1014, 1036, 610.0, 3.0, 0.0, 19.31328723350297, 175.97622871827818, 965.6643616751484, 27.89160904187871, 7.5, 4033.4066320880684], [1037, 1047, 1014, 610.0, 95.0, 2.0, 27.556461020846367, 150.7793125602313, 976.2811300657684, 34.93786667117521, 8.2123353678124, 16545.619642659498], [1040, 1014, 1037, 610.0, 3.0, 270.0, 23.878184861446904, 152.3063891659763, 832.9441898997413, 20.70612203492992, 7.5, 16959.101592923293], [1021, 1014, 1037, 322.0, 3.0, 270.0, 32.61911846754696, 134.97668520804098, 841.5667743080799, 24.78916935770

In [38]:
sdf = pd.read_csv(r'C:/Users/Penelope/Documents/GitHub/Weplate/backend/backend/algorithm/result/student_profile/nutrient_limit_female.csv')
for i in range(len(sdf)):
    b_protein, b_carbohydrate, b_calories, b_total_fat, b_saturated_fat  = sdf.loc[i, ['protein_lim', 'carbohydrate_lim', 'calories_lim', 'total_fat_lim', 'saturated_fat_lim']].values
    bound_protein = [eval(b_protein)]
    bound_carbohydrate = [eval(b_carbohydrate)]
    bound_calories = [eval(b_calories)]
    bound_total_fat = [eval(b_total_fat)]
    bound_saturated_fat = [eval(b_saturated_fat)]

with Pool(8) as p:
    inputs = list(itertools.product(bound_protein, bound_carbohydrate, bound_calories, bound_total_fat, bound_saturated_fat, combinations))
    result = p.map(pick_meal, inputs)

result = sorted(result, key=lambda x : x[-1])
print(result)

[[1238, 1208, 1236, 610.0, 79.0, 183.0, 35.807676917961466, 130.6969640612936, 984.5526118540811, 36.77560428536809, 7.963210306524423, 0], [1238, 1236, 1146, 579.0, 167.0, 150.0, 30.584814107224375, 131.3760630294013, 959.3893181395506, 36.78969347557779, 6.986829424991969, 0], [1223, 1240, 1077, 610.0, 270.0, 2.0, 34.017414240733885, 121.63183256014831, 871.8203797808953, 34.44979458205878, 6.0, 0], [1223, 1266, 1085, 610.0, 270.0, 2.0, 32.778230512057874, 129.71836273074024, 893.7795662146958, 33.602448140142876, 7.0, 0], [1223, 1266, 1068, 609.0, 268.0, 2.0, 30.743007536530936, 131.44714581919138, 852.4974499055324, 31.583427733355755, 5.0, 0], [1223, 1182, 1068, 610.0, 270.0, 2.0, 32.29986305470585, 121.06693493220438, 834.7591594315961, 35.02612136110083, 5.0, 0], [1223, 1149, 1068, 465.0, 270.0, 2.0, 30.461223731950973, 126.4877508636392, 882.3101763776882, 36.83306000491244, 5.0, 0], [1240, 1238, 1085, 343.0, 162.0, 2.0, 30.377432498689707, 121.01546993085026, 879.9158029993068

In [204]:
for c in combinations:
    print(c[0].pk, c[1].pk, c[2].pk)
    print(c[0].volume, c[1].volume, c[2].volume)
    print(c[0].normalized_nutrients())
    if c[0].normalized_nutrients()[-1] == 0:
        print(1)
    pick_meal(c)

1208 1236 1238
709.764 118.294 118.294
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1236 1223
709.764 118.294 157.7253333
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1236 1240
709.764 118.294 118.294
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1236 1266
709.764 118.294 177.441
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1236 1293
709.764 118.294 118.294
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 15

1208 1293 1085
709.764 118.294 -1.0
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (0, 2))
[300. 150.   1.]
1208 1293 1077
709.764 118.294 -1.0
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (0, 2))
[300. 150.   1.]
1208 1293 1068
709.764 118.294 -1.0
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (0, 2))
[300. 150.   1.]
1208 1276 1156
709.764 236.588 118.294
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1276 1182
709.764 236.588 118.294
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1276 114

1208 1172 1110
709.764 177.441 88.7205
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1172 1123
709.764 177.441 177.441
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1208 1172 1102
709.764 177.441 -1.0
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (0, 2))
[300. 150.   1.]
1208 1172 1119
709.764 177.441 -1.0
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (0, 2))
[300. 150.   1.]
1208 1172 1085
709.764 177.441 -1.0
[7.46727081e-02 2.25427043e-02 6.48102750e-01 2.81783804e-02
 1.26802712e-02 1.20800000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (0, 2))
[300. 150.   1.]
1208 1172 107

1236 1238 1266
118.294 118.294 177.441
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1238 1293
118.294 118.294 118.294
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1238 1276
118.294 118.294 236.588
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1238 1156
118.294 118.294 118.294
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1238 1182
118.294 118.294 118.294
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 1

1236 1266 1146
118.294 177.441 118.294
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1266 1149
118.294 177.441 118.294
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1266 1172
118.294 177.441 177.441
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1266 1110
118.294 177.441 88.7205
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 100.]
1236 1266 1123
118.294 177.441 177.441
[5.07210848e-02 1.69070283e-01 2.28244881e+00 1.60616768e-01
 3.80408136e-02 1.23600000e+03 0.00000000e+00]
1
((300, 610), (50, 270), (50, 270))
[300. 150. 1

...

1077 1238 1293
-1.0 118.294 118.294
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1276
-1.0 118.294 236.588
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1156
-1.0 118.294 118.294
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1182
-1.0 118.294 118.294
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1146
-1.0 118.294 118.294
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1149
-1.0 118.294 118.294
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1172
-1.0 118.294 177.441
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1238 1110
-1.0 118.294 88.7205
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50,

[  1. 150. 100.]
1077 1156 1149
-1.0 118.294 118.294
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1156 1172
-1.0 118.294 177.441
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1156 1110
-1.0 118.294 88.7205
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1156 1123
-1.0 118.294 177.441
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1077 1156 1102
-1.0 118.294 -1.0
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (0, 2))
[  1. 150.   1.]
1077 1156 1119
-1.0 118.294 -1.0
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (0, 2))
[  1. 150.   1.]
1077 1156 1085
-1.0 118.294 -1.0
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (0, 2))
[  1. 150.   1.]
1077 1156 1068
-1.0 118.294 -1.0
[  12.   32.  240.    8.    3. 1077.    2.]
((0, 2), (50, 270), (0, 2))


-1.0 709.764 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (0, 2))
[  1. 150.   1.]
1068 1208 1085
-1.0 709.764 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (0, 2))
[  1. 150.   1.]
1068 1208 1077
-1.0 709.764 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (0, 2))
[  1. 150.   1.]
1068 1236 1238
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1236 1223
-1.0 118.294 157.7253333
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1236 1240
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1236 1266
-1.0 118.294 177.441
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1236 1293
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2.

1068 1293 1276
-1.0 118.294 236.588
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1156
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1182
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1146
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1149
-1.0 118.294 118.294
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1172
-1.0 118.294 177.441
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1110
-1.0 118.294 88.7205
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (50, 270), (50, 270))
[  1. 150. 100.]
1068 1293 1123
-1.0 118.294 177.441
[  10.    34.   250

1068 1102 1119
-1.0 -1.0 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (0, 2), (0, 2))
[1. 1. 1.]
1068 1102 1085
-1.0 -1.0 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (0, 2), (0, 2))
[1. 1. 1.]
1068 1102 1077
-1.0 -1.0 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (0, 2), (0, 2))
[1. 1. 1.]
1068 1119 1085
-1.0 -1.0 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (0, 2), (0, 2))
[1. 1. 1.]
1068 1119 1077
-1.0 -1.0 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (0, 2), (0, 2))
[1. 1. 1.]
1068 1085 1077
-1.0 -1.0 -1.0
[  10.    34.   250.    10.     2.5 1068.     2. ]
((0, 2), (0, 2), (0, 2))
[1. 1. 1.]


In [210]:
ccc = [1156, 1149, 1068]
p = dish[dish.pk == ccc[0]]['protein'].values/dish[dish.pk == ccc[0]]['portion_volume'].values*469.5 +\
dish[dish.pk == ccc[1]]['protein'].values/dish[dish.pk == ccc[1]]['portion_volume'].values*202+\
dish[dish.pk == ccc[2]]['protein'].values/dish[dish.pk == ccc[2]]['portion_volume'].values*-1*2


carbs = dish[dish.pk == ccc[0]]['carbohydrate'].values/dish[dish.pk == ccc[0]]['portion_volume'].values*469.5 +\
dish[dish.pk == ccc[1]]['carbohydrate'].values/dish[dish.pk == ccc[1]]['portion_volume'].values*202+\
dish[dish.pk == ccc[2]]['carbohydrate'].values/dish[dish.pk == ccc[2]]['portion_volume'].values*-1*2

calo = dish[dish.pk == ccc[0]]['calories'].values/dish[dish.pk == ccc[0]]['portion_volume'].values*469.5 +\
dish[dish.pk == ccc[1]]['calories'].values/dish[dish.pk == ccc[1]]['portion_volume'].values*202+\
dish[dish.pk == ccc[2]]['calories'].values/dish[dish.pk == ccc[2]]['portion_volume'].values*-1*2

tfat = dish[dish.pk == ccc[0]]['total_fat'].values/dish[dish.pk == ccc[0]]['portion_volume'].values*469.5 +\
dish[dish.pk == ccc[1]]['total_fat'].values/dish[dish.pk == ccc[1]]['portion_volume'].values*202+\
dish[dish.pk == ccc[2]]['total_fat'].values/dish[dish.pk == ccc[2]]['portion_volume'].values*-1*2

print(p, carbs, calo, tfat)

[47.228769] [171.28503559] [1129.95587266] [33.91448425]


In [152]:
[ 45.81006644,  180.86883527, 1126.54961367,   26.29986305,8]

47.232

In [193]:
itemm = [1156, 1149, 1068]
cc = []
for iii in itemm:
    protein, carbohydrate, calories, total_fat, saturated_fat, portion_volume, max_pieces = dish[dish.pk == iii][['protein', 'carbohydrate', 'calories', 'total_fat', 'saturated_fat', 'portion_volume', 'max_pieces']].values.squeeze(0)
    mm = dish_nutrient(protein, carbohydrate, calories, total_fat, saturated_fat, portion_volume, iii, max_pieces)
    cc.append(mm)


In [194]:
print(pick_meal(cc))

((100, 610), (50, 270), (0, 2))
[300. 150.   1.]
[1156, 1149, 1068, 158.0, 270.0, 2.0, array([ 32.57882902, 126.66738803, 865.69902108,  30.65988131,
         5.        ]), 95310.03029916059]
